# Gradio Chatbot UI for Multi-Agent System
Interact with your AI agents using a conversational interface powered by Gradio and the logic in `mas.py`.

In [ ]:
import asyncio
import gradio as gr
from src.mas import Orchestrator

# Initialize the orchestrator
orchestrator = Orchestrator()

# Global chat history
chat_history = []

## Define the Chatbot Logic
This function will be called on each user message. It sends the message to the orchestrator and returns the updated chat history, clears the input, and optionally a filename to plot.

In [ ]:
def chat_with_agents(user_message, history):
    global chat_history
    file_name = None
    if not user_message.strip():
        return history, "", None
    # Append user message to history for display
    history = history or []
    history.append(("You", user_message))
    # Run orchestrator and get agent responses
    try:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        results, file_name = loop.run_until_complete(orchestrator.run(user_message))
        # Append each response from the orchestrator
        for result in results:
            role = result.get("role", "")
            name = result.get("name", "")
            content = result.get("content", "")
            history.append((name or role, content))
            # file_name is obtained from orchestrator return value
    except Exception as e:
        history.append(("system", f"Error: {e}"))
    return history, "", file_name

## Build the Gradio UI
This UI includes a chatbot window, a textbox for user input, and an image component to display any returned image file.

In [ ]:
import debugpy

# Allow other clients to attach to debugpy
debugpy.listen(("0.0.0.0", 5678))
print("Waiting for debugger to attach...")

# Pause execution until the debugger is attached
debugpy.wait_for_client()

with gr.Blocks(title="Reasoning over ML - Chatbot") as demo:
    gr.Markdown("""# Multi-Agent Chatbot\nChat with your AI agents below.""")
    chatbot = gr.Chatbot(label="Conversation", height=700)
    textbox = gr.Textbox(label="Your Message", placeholder="Type your message and press Enter", show_label=True, lines=1)
    image = gr.Image(type="filepath", label="Generated Image")

    def clear_history():
        global chat_history
        chat_history = []
        return [], None

    textbox.submit(chat_with_agents, [textbox, chatbot], [chatbot, textbox, image])
    gr.Button("Clear").click(clear_history, outputs=[chatbot, image])

if __name__ == "__main__":
    demo.launch()